In [1]:
# import all dependencies
import pandas as pd
import numpy as np
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cross_validation import cross_val_score
from scipy.sparse import hstack

/Users/bilindhajer/Desktop/atl-dat2/lessons/lesson-13/code/solution-code


In [79]:
# put your functions in one cell, call when you need
def read_tsv(path):
    return pd.read_csv(path, sep='\t')
def get_json_value(col,key):
    return json.loads(col).get(key, '')
def has_text(text,text_val):
    try:
        if 'recipe' in str(text).lower():
            return 1
        else:
            return 0
    except: 
        return 0

In [80]:
path = "../../assets/dataset/stumbleupon.tsv"
data = read_tsv(path)

In [81]:
data.dtypes

url                                object
urlid                               int64
boilerplate                        object
alchemy_category                   object
alchemy_category_score             object
avglinksize                       float64
commonlinkratio_1                 float64
commonlinkratio_2                 float64
commonlinkratio_3                 float64
commonlinkratio_4                 float64
compression_ratio                 float64
embed_ratio                       float64
framebased                          int64
frameTagRatio                     float64
hasDomainLink                       int64
html_ratio                        float64
image_ratio                       float64
is_news                            object
lengthyLinkDomain                   int64
linkwordscore                       int64
news_front_page                    object
non_markup_alphanum_characters      int64
numberOfLinks                       int64
numwords_in_url                   

In [100]:
data.label.head(5)

0    0
1    1
2    1
3    1
4    0
Name: label, dtype: int64

In [96]:
# get the title text from the JsonObject
boiler_plate_col = data.boilerplate

data['title'] = boiler_plate_col.apply(
    lambda x: get_json_value(x,'title')
)

In [101]:
# get the body text from the JsonObject
data['body'] = boiler_plate_col.apply(
    lambda x: get_json_value(x,'body')
)

In [102]:
# Let's try extracting some of the text content.
# Create a feature for the title containing 'recipe'. 
# Is the % of evegreen websites higher or lower on pages 
# that have recipe in the the title?

data['has_recipe'] = data.title.apply(
    lambda text: has_text(text,"recipe")
)

In [103]:
print data[ data.has_recipe == 1 ].shape

(926, 30)


In [104]:
print data[ data.has_recipe == 0 ].shape

(6469, 30)


In [105]:
#The majority of the websites do not have the word 'recipe' in there titles

In [106]:
# remove all null values for title
data['title'] = data['title'].fillna('')

In [2]:
# lets make every word in the title a feature, 
# maybe other words besides recipes contributed to 
# higher evergreen websites?

vectorizer = CountVectorizer(
                max_features = 1000, # max number of words to consider as features
                ngram_range=(1, 2), 
                stop_words='english', # stop when you see word english
                binary=True # each word will be a binary dummy categorical var
            )

In [3]:
# Use `fit` to learn the vocabulary of the titles
vectorizer.fit(data.title)

NameError: name 'data' is not defined

In [4]:
# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
x_text_features = vectorizer.transform(data.title)

NameError: name 'data' is not defined

In [110]:
labels = data.label

In [111]:
model = RandomForestClassifier(n_estimators = 20)

In [112]:
scores = cross_val_score(
            model, 
            x_text_features, 
            labels, 
            scoring='roc_auc'
        )
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

CV AUC [ 0.79372499  0.80660573  0.80162289], Average AUC 0.800651203991


In [113]:
# Identify the features you want from the original dataset
other_features_columns = ['html_ratio', 'image_ratio']
other_features = data[other_features_columns]

In [114]:
# Stack them horizontally together
# This takes all of the word/n-gram columns and appends on two more columns for `html_ratio` and `image_ratio`
features = hstack((x_text_features, other_features)).toarray()

scores = cross_val_score(model, features, labels, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

CV AUC [ 0.78873782  0.79437385  0.79370226], Average AUC 0.792271310491


In [115]:
# What features of these are most important?
model.fit(features, labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [116]:
all_feature_names = vectorizer.get_feature_names() + other_features_columns

In [117]:
feature_importances = pd.DataFrame(
    {'Features' : all_feature_names, 
     'Importance Score': model.feature_importances_
    }
)

In [118]:
feature_importances.sort_values(
   'Importance Score', 
    ascending=False
).head(10)

,Features,Importance Score
1000,html_ratio,0.154404
1001,image_ratio,0.092784
712,recipe,0.034786
721,recipes,0.020107
190,chocolate,0.011673
180,chicken,0.011642
150,cake,0.009957
496,kitchen,0.009686
342,fashion,0.009553
611,news,0.008109
